<a href="https://colab.research.google.com/github/victorjms/prediccion-ventas/blob/main/Proyecto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>